In [1]:
#python resample.py $IMAGE_TO_RESAMPLE $REFERENCE_IMAGE
"""
8-3: implemented exclusion criteria based on percent TRs and whether or not there was no bpress
8-16: destroyed everything that wasn't necessary for creating current head motion criteria
"""

"\n8-3: implemented exclusion criteria based on percent TRs and whether or not there was no bpress\n8-16: destroyed everything that wasn't necessary for creating current head motion criteria\n"

In [2]:
#jupyter nbconvert --to python slurm_create-data_preproc.ipynb

In [3]:
import nibabel as nib

from nilearn.input_data import NiftiMasker , MultiNiftiMasker
import nilearn as nil
import numpy as np 
import os
import os.path
import scipy.io
import nibabel as nib
from nilearn.input_data import NiftiMasker
from nilearn.masking import compute_epi_mask
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import PredefinedSplit
from copy import deepcopy
import warnings
import sys  
import random
# import logging

import deepdish as dd
import numpy as np

import brainiak.eventseg.event
import nibabel as nib
from nilearn.input_data import NiftiMasker

import scipy.io
from scipy import stats
from scipy.stats import norm, zscore, pearsonr
from scipy.signal import gaussian, convolve
from sklearn import decomposition
from sklearn.model_selection import LeaveOneOut, KFold

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches
import seaborn as sns 

#%autosave 5
#%matplotlib inline
sns.set(style = 'white', context='talk', font_scale=1, rc={"lines.linewidth": 2})

if not sys.warnoptions:
    warnings.simplefilter("ignore")


random.seed(10)
#%matplotlib inline
from brainiak import image, io
from scipy.stats import stats
import nibabel as nib
import numpy as np
from matplotlib import pyplot as plt
from brainiak import image, io
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA, NMF
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import LeavePGroupsOut
import pandas as pd
# Import machine learning libraries
from nilearn.input_data import NiftiMasker
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold, f_classif, SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from scipy.stats import sem
from copy import deepcopy
from sklearn.metrics import roc_auc_score
import statistics
# Visualize it as an ROI
from nilearn.plotting import plot_roi
#plot_roi(x)
from nilearn.image import concat_imgs, resample_img, mean_img
from nilearn.plotting import view_img
from nilearn import datasets, plotting
from nilearn.input_data import NiftiSpheresMasker

from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.image import concat_imgs, resample_img, mean_img,index_img
from nilearn import image
from nilearn import masking
from nilearn.plotting import view_img
from nilearn.image import resample_to_img
import math
import pandas as pd
import os

# Functions 

In [4]:
def load_epi_data(sub, ses, task,run, space):
  # Load MRI file
    if space == "MNI":
        epi_in = os.path.join(data_dir, sub, ses, 'func', "%s_%s_task-%s_run-%s_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz" % (sub, ses, task,run))
    elif space == "T1":
        epi_in = os.path.join(data_dir, sub, ses, 'func', "%s_%s_task-%s_run-%s_space-T1w_desc-preproc_bold.nii.gz" % (sub, ses, task,run))
    else:
        print("wrong load epi input. check this function")
    epi_data = nib.load(epi_in)
    print(epi_data.shape)
    print("Loading data from %s" % (epi_in))
    return epi_data

def load_roi_mask(ROI_name, space):
    if space == "MNI":
        maskdir = os.path.join(rois_dir)    
        print("expected shape: 78, 93,65")
    elif space == "T1":
        maskdir = os.path.join(rois_dir+ "/T1")
        print("expected shape: 56, 72,53")
    else:
        print("wrong mask input. check this function")
    # load the mask
    maskfile = os.path.join(maskdir, "%s.nii" % (ROI_name))
    mask = nib.load(maskfile)
    print("mask shape: ", mask.shape)
    print("Loaded %s mask" % (ROI_name))
    return mask
def intersect_mask(sub, num_runs,reg, ses="ses-01",task="Attn"):
    # This is based off of 'load_data' function in template
    # Loads all fMRI runs into a matrix #
    """
    reg = T1 or MNI registration?
    norm_type = by Space or by Time? 
    """
    yoz = []
    print("Begin intersecting, you sexy beast")
    for run in range(1, num_runs + 1):
        if sub == "sub-002":
            if run >=7:
                run = run+1
        # Load epi data 
        epi = load_epi_data(sub,ses,task,run,reg)
        # Mask data
        roi_samp = compute_epi_mask(epi) # -- whole brain
        #roi_samp load_roi_mask(ROI_name,reg) # -- mask

        nifti_masker = NiftiMasker(mask_img=roi_samp)
        maskedData = nifti_masker.fit_transform(epi)
        yoz.append(roi_samp)
    #print(concatenated_data)
    epi_data = nil.masking.intersect_masks(yoz)
    print("all done wit da intersextion (lol)")

    return epi_data




def label_lists(label, num_tr):
    b = [[]]
    a = []
    for i in label:
        # substring label in psychopy output
        # if the first three characters == M_s, etc, then add correct indext to string
        if i[1:4] == "M_s":
            a.append("SM")
            b.append([0]*num_tr)
        elif i[1:4] == "C_s":
            a.append("SC")
            b.append([1]*num_tr)        
        elif i[1:4] == "M_o":
            a.append("OM")
            b.append([2]*num_tr)
        elif i[1:4] == "C_o":
            a.append("OC")
            b.append([3]*num_tr)     
        else:
            a.append("Re")
            b.append([4]*num_tr)     
    return a, b[1:]


In [5]:
def load_fMRI3d(sub, num_runs,reg, norm_type, ses="ses-01",task="Attn"):
    # This is based off of 'load_data' function in template
    # Loads all fMRI runs into a matrix #
    """
    reg = T1 or MNI registration?
    norm_type = by Space or by Time? - default is by space (rows)
    """
    concatenated_data = []
    int_mask = intersect_mask(sub, 1,reg)
    # 
    for run in range(1, num_runs + 1):
        if sub == "sub-002":
            if run >=7:
                run = run+1
        if sub != "sub-010":
            # Load epi data 
            epi = load_epi_data(sub,ses,task,run,reg)
        else:
            # Load epi data 
            print("sub-10, watch out")
            bad_epi = load_epi_data(sub,ses,task,run,reg)
            good_epi = load_epi_data("sub-001",ses,task,run,reg)
            epi = resample_to_img(bad_epi , good_epi, interpolation='nearest')
            int_mask = resample_to_img(int_mask , good_epi, interpolation='nearest')
        # delete first 9 TRs
        epi = index_img(epi,slice(9,209))
        
        # load confounds
        run_conf = np.asarray(pd.read_csv(os.path.join(confounds + sub + "/func/", 
                                                           '%s_ses-01_task-Attn_run-%s_desc-model_timeseries.csv') % (sub, run)))
        print(run_conf.shape)
        # clean image
        # low_pass= .1,
        clean_bold = image.clean_img(epi, standardize = False, confounds = run_conf[9:], high_pass=1/128, 
                                   t_r=1.5, mask_img = int_mask)
        
        
        #Smooth
        clean_bold = image.smooth_img(clean_bold, fwhm=5)
        
        # F*k it mask off -- Load ROI data
        #roi_samp =load_roi_mask(ROI_name,reg)
        # Pull voxels from epi data # *** may need to change this 
        #nifti_masker = NiftiMasker(mask_img=roi_samp)
        #masked_data = nifti_masker.fit_transform(clean_bold)
        
        #append to cat_dat
        concatenated_data.append(clean_bold)
    "FINISHED YAY BEAST"
    return concatenated_data, epi

In [6]:
def load_fMRI3d(sub, num_runs,reg, norm_type, mask, ses="ses-01",task="Attn"):
    # This is based off of 'load_data' function in template
    # Loads all fMRI runs into a matrix #
    """
    reg = T1 or MNI registration?
    norm_type = by Space or by Time? - default is by space (rows)
    """
    concatenated_data = []
    
    for run in range(1, num_runs + 1):
        if sub == "sub-002":
            if run >=7:
                run = run+1
        if sub != "sub-010":
            # Load epi data 
            epi = load_epi_data(sub,ses,task,run,reg)
        else:
            # Load epi data 
            print("sub-10, watch out")
            bad_epi = load_epi_data(sub,ses,task,run,reg)
            good_epi = load_epi_data("sub-001",ses,task,run,reg)
            epi = resample_to_img(bad_epi , good_epi, interpolation='nearest')
        # delete first 9 TRs
        epi = index_img(epi,slice(9,209))
        
        # load confounds
        run_conf = np.asarray(pd.read_csv(os.path.join(confounds + sub + "/func/", 
                                                           '%s_ses-01_task-Attn_run-%s_desc-model_timeseries.csv') % (sub, run)))
        print(run_conf.shape)
        # clean image
        # low_pass= .1,
        clean_bold = image.clean_img(epi, standardize = False, confounds = run_conf[9:], high_pass=1/128, 
                                   t_r=1.5, mask_img = mask)
        
        
        #Smooth
        clean_bold = image.smooth_img(clean_bold, fwhm=5)
        
        # F*k it mask off -- Load ROI data
        #roi_samp =load_roi_mask(ROI_name,reg)
        # Pull voxels from epi data # *** may need to change this 
        #nifti_masker = NiftiMasker(mask_img=roi_samp)
        #masked_data = nifti_masker.fit_transform(clean_bold)
        
        #append to cat_dat
        concatenated_data.append(clean_bold)
    "FINISHED YAY BEAST"
    return concatenated_data, epi

In [7]:
def label_indices(behav_p, sub):
    behav = pd.read_csv(os.path.join(behav_p, '%s_behav_cleaned.csv') % (sub))
    # Define the column in behav to be used for creating labels # 
    label = behav.iloc[:,1]
    # Create an array of labels [1] AND the order in which runs occured [0]#
    sub_ses_labels = label_lists(label, 200)
    ## Find run sequence, extraction condition indexes from behav data ## 
    return find_cond_index(sub_ses_labels[0])

def find_cond_index(sub_ses_labels):
    """
    For the array of ordered run names (i.e.'Re', 'SM',) find the two indexes per condition
    """ 
    lab_inx = []

    a = []
    b = []
    c = []
    d = []

    for i in enumerate(sub_ses_labels):
        if i[1] == "SM":
            # append the index according to where it appeared in the array
            a.append(i[0])
        if i[1] == "SC":
            b.append(i[0])
        if i[1] == "OM":
            c.append(i[0])
        if i[1] == "OC":
            d.append(i[0])

    # Create a dictionary where each key contains the appropriate indexes
    lab_indic = {
        'SM' : a,
        'SC' : b,
        'OM' : c,
        'OC' : d,
        'RE' : [0,9]
    }
    return lab_indic 
    #np.vstack(lab_inx, ["SM", "SC", "OM", "OC"])
    

def load_confounds(cond_list, sub_list,behav_p,confounds):
    """
    args: 
        cond_list: list of conditions (cond_list=np.array(['SM','SC']))
        sub_list: subjects to extract confounds for
        behav_p: path to the behavioral data
        confounds: path to the confound data
    returns:
        nested dictionary in the form of: conf_sub[sub][cond][img_ind]
        where img_index is the first or second run
    """
    # Confound files

    conf_sub = {}
    for sub in sub_list:
        conf_cond = {}
        for cond in cond_list:
            confs = []
            lab_indic = fnd_indices(sub, behav_p)
            ## plus one because indices are 0 thru 9 ## 
            run_1_ind = lab_indic[cond][0] +1
            run_2_ind = lab_indic[cond][1] +1
            if sub == 'sub-002':
                if run_1_ind >=7 : run_1_ind +=1
                if run_2_ind >=7 : run_2_ind +=1
            
            # Confound load #    
            confs.append(np.asarray(pd.read_csv(os.path.join(confounds + sub + "/func/", 
                                                             '%s_ses-01_task-Attn_run-%s_desc-model_timeseries.csv') % (sub, run_1_ind)))[4:,:])
            confs.append(np.asarray(pd.read_csv(os.path.join(confounds + sub + "/func/",
                                                             '%s_ses-01_task-Attn_run-%s_desc-model_timeseries.csv') % (sub, run_2_ind)))[4:,:])
            conf_cond[cond] = confs
        conf_sub[sub] = conf_cond
    return conf_sub



In [8]:
def fnd_indices(sub,behav_p):
    behav = pd.read_csv(os.path.join(behav_p, '%s_behav_cleaned.csv') % (sub))
    # Define the column in behav to be used for creating labels # 
    label = behav.iloc[:,1]
    # Create an array of labels [1] AND the order in which runs occured [0]#
    sub_ses_labels = label_lists(label, 200)
    ## Find run sequence, extraction condition indexes from behav data ## 
    return find_cond_index(sub_ses_labels[0])

# Define Static VARS

In [9]:
####
data_dir = "/jukebox/graziano/coolCatIsaac/ATM/data/bids/derivatives/fmriprep/"
rois_dir = "/jukebox/graziano/coolCatIsaac/ATM/data/work/rois/"
behav_p = '/jukebox/graziano/coolCatIsaac/ATM/data/behavioral'
sav_work = "/jukebox/graziano/coolCatIsaac/ATM/data/work/results/corr_data/"
load_work = "/jukebox/graziano/coolCatIsaac/ATM/data/work/results/corr_data/"
confounds = '/jukebox/graziano/coolCatIsaac/ATM/data/bids/derivatives/fmriprep/afni-head_mot/'
workspace = "/jukebox/graziano/coolCatIsaac/ATM/data/work/workspace/"
parc_dir = "/jukebox/graziano/coolCatIsaac/ATM/data/work/rois/schaef_par/MNI/"
sav_fcma = '/jukebox/graziano/coolCatIsaac/ATM/data/work/workspace/load_fcma'
censor_dir = '/jukebox/graziano/coolCatIsaac/ATM/data/work/workspace/censor_hm'
utils = '/jukebox/graziano/coolCatIsaac/ATM/code/analysis/bpress/utils.py'
load_bpress = "/jukebox/graziano/coolCatIsaac/ATM/data/work/results/bpress_GLM/behav"
confounds_dir = '/jukebox/graziano/coolCatIsaac/ATM/data/work/workspace/censor_hm/'


In [10]:
censor_dir

'/jukebox/graziano/coolCatIsaac/ATM/data/work/workspace/censor_hm'

In [11]:
#from utils import fnd_indices

# lesss do it

In [13]:
##### GENERAL ####
sub_list = ["sub-000","sub-001","sub-003","sub-004","sub-005","sub-006","sub-007","sub-008","sub-009",
            "sub-011","sub-012","sub-013","sub-014"]
sub_list = ["sub-000","sub-001","sub-005","sub-006","sub-007","sub-008","sub-009",
            "sub-011","sub-012","sub-013","sub-014"]
sub_list = ["sub-015", "sub-016","sub-017", "sub-018", "sub-019", "sub-020","sub-021", "sub-021"]
sub_list = ["sub-000","sub-001","sub-002","sub-003","sub-004","sub-005","sub-006","sub-007","sub-008","sub-009",
            "sub-010","sub-011","sub-012","sub-013","sub-014","sub-015", "sub-016","sub-017", 
            "sub-018", "sub-019", "sub-020","sub-021"]
"""
sub_list = ["sub-011","sub-012","sub-013","sub-014","sub-015", "sub-016","sub-017", 
            "sub-018", "sub-019", "sub-020","sub-021"]
            
"""
sub_list = ["sub-000","sub-001","sub-002","sub-003","sub-004","sub-005","sub-006","sub-007","sub-008","sub-009",
            "sub-010","sub-011","sub-012","sub-013","sub-014","sub-015", "sub-016","sub-017", 
            "sub-018", "sub-019", "sub-020","sub-021",'sub-022','sub-023','sub-024','sub-025','sub-026','sub-027']
#sub_list = ["sub-005"]

# censor threshold
num_runs = 10
### Change me ## 
fd = .3
thresh= str(3)


# Find the censor matrix of ones for each censored TR 

In [14]:
def find_censTR_mat(cens_tr, tr_len=209):
    """
    args: 
        cens_tr: indices of censored TRs
    """
    if cens_tr.shape[0] == 0:
        return cens_tr
    else:
        cens_mat = np.zeros((tr_len, int(len(cens_tr))))
        for idx, tr in enumerate(cens_tr):
            cens_col = np.zeros((tr_len))
            cens_col[tr] = 1
            cens_mat[:,idx] = cens_col
        print("censoring ", cens_mat.shape[1], 'trs')
    return cens_mat
    

In [15]:
################## censor me timbers ##################
"""
fd_arr: array of ones and zeros, with ones being TRs above the threshold, length 209
tot_cens: number of TRs censored per that run
t_dic = fd_arr sorted by condition in a dictionary 
all_subs = dictionary of sorted sub censored TR data
"""
head_mot_dic = {}
fd_arr = []
cond_list = ['SM', "SC", "RE", 'OM', 'OC']
all_subs = {}
run_cens_list = []

for sub in sub_list:
    temp_dic = {}
    t_dic = {}
    lab_indic = label_indices(behav_p, sub)
    for run in range(1,num_runs+1):
        if sub == "sub-002":
            if run >=7:
                run = run+1
        confounds_fn = pd.read_csv(os.path.join(data_dir, sub + "/ses-01/func","%s_ses-01_task-Attn_run-%s_desc-confounds_timeseries.tsv" % (sub, run)),sep='\t')
        print(len(confounds_fn[confounds_fn['framewise_displacement'] > fd]['framewise_displacement']))
        # Find percentage that were censored for each run at the current threshold
        #tot_cens = len(confounds_fn[confounds_fn['framewise_displacement'] > fd]['framewise_displacement']) /209
        #Find average framewsie displacement for the current run
        tot_cens = np.nanmean(np.asarray(confounds_fn['framewise_displacement']))
        # creat binary array
        bin_arr = np.asarray(confounds_fn['framewise_displacement'] > fd).astype(int)
        # find indices of censored TRs
        cens_tr = np.where(bin_arr ==1)[0]
        cens_mat = find_censTR_mat(cens_tr, tr_len=209)
        run_cens_list.append(cens_mat) 
    # Above we aggregate the num censored into a list of 10. BELOW we SORT!
    for cond in cond_list:
        #print(cond)
        run1 = run_cens_list[lab_indic[cond][0]]
        run2 = run_cens_list[lab_indic[cond][1]]
        t_dic[cond+"-1"] = run1
        t_dic[cond+"-2"] = run2
    all_subs[sub] = t_dic
## only need to save once. all_subs will be used below
#np.save(os.path.join(censor_dir, 'n28_numtrs_censored_at_threshp3.npy'),all_subs)
        

27
censoring  27 trs
36
censoring  36 trs
45
censoring  45 trs
23
censoring  23 trs
5
censoring  5 trs
9
censoring  9 trs
34
censoring  34 trs
43
censoring  43 trs
16
censoring  16 trs
12
censoring  12 trs
3
censoring  3 trs
9
censoring  9 trs
10
censoring  10 trs
31
censoring  31 trs
23
censoring  23 trs
9
censoring  9 trs
34
censoring  34 trs
40
censoring  40 trs
29
censoring  29 trs
12
censoring  12 trs
9
censoring  9 trs
61
censoring  61 trs
32
censoring  32 trs
19
censoring  19 trs
7
censoring  7 trs
36
censoring  36 trs
12
censoring  12 trs
14
censoring  14 trs
82
censoring  82 trs
11
censoring  11 trs
8
censoring  8 trs
6
censoring  6 trs
32
censoring  32 trs
34
censoring  34 trs
13
censoring  13 trs
30
censoring  30 trs
31
censoring  31 trs
46
censoring  46 trs
33
censoring  33 trs
49
censoring  49 trs
5
censoring  5 trs
0
10
censoring  10 trs
14
censoring  14 trs
19
censoring  19 trs
17
censoring  17 trs
23
censoring  23 trs
8
censoring  8 trs
18
censoring  18 trs
6
censoring 

# Merge Sams HM matrix with censored @ .3 matrix

In [16]:
# now create a merged file for loading into the glm using data from above 
"""
all_subs: censored matrix of ones and zeroes, length 205
og_conf: sam's HM matrix (all confounds that we would normally insert)
output, sub_dic is a full HM merged list of sams confounds + TRs to enclude
"""


"\nall_subs: censored matrix of ones and zeroes, length 205\nog_conf: sam's HM matrix (all confounds that we would normally insert)\noutput, sub_dic is a full HM merged list of sams confounds + TRs to enclude\n"

In [17]:
#sub_list = ['sub-000']
og_conf = load_confounds(cond_list, sub_list,behav_p, confounds)

In [18]:
full_conf_sub = {}
for sub in sub_list:
    cond_dic = {}
    for cond in cond_list:
        run_arr = []
        for run in np.arange(2):
            # if no censoring, return old matrix
            if all_subs[sub][cond+'-'+str(run+1)].shape[0] != 0:
                og_conf_len = og_conf[sub][cond][run].shape[1]
                cens_conf_len = all_subs[sub][cond+'-'+str(run+1)][4:,:].shape[1]
                col_len =  og_conf_len + cens_conf_len
                new_dat = np.zeros((205,col_len))
                new_dat[:,:og_conf_len] = og_conf[sub][cond][run]
                new_dat[:,og_conf_len:] = all_subs[sub][cond+'-'+str(run+1)][4:,:]
                run_arr.append(new_dat)
            else:
                run_arr.append(og_conf[sub][cond][run])
        cond_dic[cond] = run_arr
    full_conf_sub[sub] = cond_dic 
    print(sub)

sub-000
sub-001
sub-002
sub-003
sub-004
sub-005
sub-006
sub-007
sub-008
sub-009
sub-010
sub-011
sub-012
sub-013
sub-014
sub-015
sub-016
sub-017
sub-018
sub-019
sub-020
sub-021
sub-022
sub-023
sub-024
sub-025
sub-026
sub-027


# Exclude runs

In [19]:
"""
How to use this part:
- not quite sure how the hell i created run_dic, but consists of all the runs to be included 
    based on which subjects had button presses: np.array([0,1]) if both runs to be included 
- full_conf_sub = full confound list to be condensed based on run exclusion
- tot_cens: n28_numtrs_censored_at_threshp3.npy: created using excel part below which sums TRs with more than .x head motion

"""

'\nHow to use this part:\n- not quite sure how the hell i created run_dic, but consists of all the runs to be included \n    based on which subjects had button presses: np.array([0,1]) if both runs to be included \n- full_conf_sub = full confound list to be condensed based on run exclusion\n- tot_cens: n28_numtrs_censored_at_threshp3.npy: created using excel part below which sums TRs with more than .x head motion\n\n'

In [20]:
### Change me ! and the files to load in below## 
## this is the confounds matrix ##
out_name = 'n28_conf+cens_MERGE_removNoBpress_delHMruns_threshp'+thresh+'_glm.npy'
## this is the runs to be included
run_dic_name = 'n28_runs_2_include_removNoBpress_delHMruns_threshp'+thresh+'.npy'
# out_name = 'test.npy'
# ** may have to change this! could be 205 if some trs are on the borderline

# what percentage trs can be excluded per run?
excl = .3
abov_thresh = int(209*excl)
cond_list = ['SM', "SC", "RE", 'OM', 'OC']

In [21]:
censor_dir

'/jukebox/graziano/coolCatIsaac/ATM/data/work/workspace/censor_hm'

In [22]:
# Load Confounds #
#conf_sub = dict(enumerate(np.load(os.path.join(confounds_dir, 'n28_conf+cens_MERGE_10r_threshp%s_glm.npy') %(thresh), 
    #                                      allow_pickle = True).flatten(),1))[1]
## full confound list is calculated above

# full_conf_sub # 

In [23]:
# Load total TRs censored #
tot_cen = dict(enumerate(np.load(os.path.join(confounds_dir, 'n28_numtrs_censored_at_threshp%s.npy')%(thresh), 
                                          allow_pickle = True).flatten(),1))[1]

In [24]:
## runs to include -- this was manually created to delete subs without button presses ## 
## we will merge this with other exclusion criteria
run_dic = dict(enumerate(np.load(os.path.join(load_bpress, "n28_runs_2_include.npy"), 
                                allow_pickle=True).flatten(),1))[1]

In [25]:
sub_list = ['sub-019']

In [26]:
## Part 1 ## 
# This loop will iterate through tot cens, which is a dictionary defining how many TRs are above a thresh
# produces a cens_dat matrix consiting of which runs to include
cens_dat = {}
for sub in sub_list:
    temp = {}
    for cond in cond_list:
        temp2= []
        for run_num in range(2):
            if tot_cen[sub][cond+"-"+str(run_num+1)] < abov_thresh:
                temp2.append(run_num)
        # save runs where the indices should be included according to run_dic
        temp[cond] = np.asarray(temp2)
    cens_dat[sub] = temp
    

In [27]:
## Part 2 ## 
# Remove runs without a button press AND runs above the censor threshold#
# produces confound matrix with no bpress runs removed and censor threshold removed
new_run_dic = {}
conf_sub2 = {}
for sub in sub_list:
    print(sub)
    temp = {}
    temp_run_dic = {}
    for cond in cond_list:
        # select only the runs where there was a button press and above the mot thresh
        intersect_arr = np.intersect1d(run_dic[sub][cond], cens_dat[sub][cond])
        temp_run_dic[cond] = intersect_arr
        # save runs where the indices should be included according to run_dic
        temp[cond] = [full_conf_sub[sub][cond][i] for i in intersect_arr]
        #print(conf_sub[sub][cond])
    conf_sub2[sub] = temp
    new_run_dic[sub] = temp_run_dic

# save confounds
#np.save(os.path.join(confounds_dir, out_name),conf_sub2)
# Save new run dic that exludes runs with 30% of data gone
#np.save(os.path.join(confounds_dir, run_dic_name),new_run_dic)

sub-019


In [28]:
new_run_dic

{'sub-019': {'SM': array([0]),
  'SC': array([0, 1]),
  'RE': array([], dtype=float64),
  'OM': array([], dtype=int64),
  'OC': array([0])}}

In [29]:
run_dic['sub-019']

{'SM': array([0, 1]),
 'SC': array([0, 1]),
 'OM': array([1]),
 'OC': array([0, 1]),
 'RE': array([0, 1])}

In [82]:
conf_sub2['sub-005']['SM'][1]

IndexError: list index out of range

In [ ]:
# this is what i used for the excel doc, not for head motion censoring for mvpa, which is above and unfinished